In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

### Import the Data

In [3]:
# Interest Rate
i = pd.read_csv('i_q.csv', sep = ';', index_col = 0)

# Rate of Inflation (% change from previous year)
pi = pd.read_csv('pi_q.csv', sep = ';', index_col = 0)

# Gross Domestic Product
y = pd.read_csv('y_q.csv', sep = ';', index_col = 0)

# Potential Output
ystar = pd.read_csv('ystar_q.csv', sep = ';', index_col = 0)

# Unemployment Rate
u = pd.read_csv('u_q.csv', sep = ';', index_col = 0)

# Residential Property Price Index
rppi = pd.read_csv('rppi_q.csv', sep = ';', index_col = 0).Index.str.replace(',', '.').astype(float)

# Collect each quarterly time series in a DataFrame
dfq = pd.DataFrame({
    'year': i.index.str[:4],
    'quarter': i.index.str[-1:],
    'month': np.nan, # will be used later
    'i' : i.Value.values/100,
    'pi': pi.Value.values/100,
    'y': y.iloc[:, 0].values,
    'ystar': ystar.iloc[:, 0].values,
    'u': u.iloc[:, 0].values / 100,
    'rppi': rppi.values
})

dfq.head()

,year,quarter,month,i,pi,y,ystar,u,rppi
0,1990,1,NaN,0.089833,0.030558,3.351394e+10,3.376665e+10,0.0341,66.0
1,1990,2,NaN,0.088600,0.030335,3.409479e+10,3.376665e+10,0.0341,74.3
2,1990,3,NaN,0.087933,0.032854,3.466919e+10,3.376665e+10,0.0341,82.5
3,1990,4,NaN,0.092200,0.036668,3.525926e+10,3.376665e+10,0.0341,83.3
4,1991,1,NaN,0.094533,0.033952,3.589568e+10,3.608909e+10,0.0342,85.5


### Interpolation

In [4]:
# Set the monthly values equal to the quarterly values
df = dfq.loc[dfq.index.repeat(3),:]

# Adjust the cumulative values (quarterly = sum of monthly)
df[['y', 'ystar']] = df[['y', 'ystar']] / 3

# Compute the Inflation Gap
df['inflationGap'] = df.pi - 0.02

# Compute the Output Gap as percentage of Potential Output
df['outputGap'] = (df.y - df.ystar) / df.ystar

# update the index
df.set_index(np.arange(df.shape[0]), inplace = True)

# Recover the month
df['month'] = df.index%12 + 1

df.head()

,year,quarter,month,i,pi,y,ystar,u,rppi,inflationGap,outputGap
0,1990,1,1,0.089833,0.030558,1.117131e+10,1.125555e+10,0.0341,66.0,0.010558,-0.007484
1,1990,1,2,0.089833,0.030558,1.117131e+10,1.125555e+10,0.0341,66.0,0.010558,-0.007484
2,1990,1,3,0.089833,0.030558,1.117131e+10,1.125555e+10,0.0341,66.0,0.010558,-0.007484
3,1990,2,4,0.088600,0.030335,1.136493e+10,1.125555e+10,0.0341,74.3,0.010335,0.009718
4,1990,2,5,0.088600,0.030335,1.136493e+10,1.125555e+10,0.0341,74.3,0.010335,0.009718


In [68]:
df.to_csv('df.csv')